## Mount the drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')


## cd to My Drive and create InceptionV4 directory, with train,validation and checkpoints folders

In [0]:
!cd drive/My\ Drive/
!mkdir InceptionV4
!cd InceptionV4
!mkdir train
!mkdir val
!mkdir checkpoints

## Install requirements

In [0]:
!pip3 install absl-py==0.7.1
!pip3 install jsonpatch==1.16
!pip3 install jsonpointer==1.10
!pip3 install jsonschema==2.6.0
!pip3 install Keras==2.2.4
!pip3 install Keras-Applications==1.0.8
!pip3 install Keras-Preprocessing==1.1.0
!pip3 install matplotlib==3.1.1
!pip3 install numpy==1.17.0
!pip3 install pandas==0.25.0
!pip3 install pickleshare==0.7.5
!pip3 install Pillow==6.1.0
!pip3 install protobuf==3.9.1
!pip3 install scikit-learn==0.21.3
!pip3 install scipy==1.3.1
!pip3 install six==1.12.0
!pip3 install sklearn==0.0
!pip3 install tensorboard==1.14.0
!pip3 install tensorflow-estimator==1.14.0
!pip3 install tensorflow-gpu==1.13.1

# InceptionV4

In [0]:
from keras.layers import Input, merge, Dropout, Dense, Flatten, Activation
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils.training_utils import multi_gpu_model
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from sklearn.utils import shuffle
import os
import random
import IPython.display as display
import scipy
from tensorflow import set_random_seed
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pathlib
import PIL
import pandas
from keras.layers import concatenate




# In case you're using pretrained model, uncomment the next line and give the direction of the checkpoint to it

#checkpoint_path = 'checkpoints/inceptionv4.h5'

def conv_block(x, nb_filter, nb_row, nb_col, border_mode='same', subsample=(1, 1), bias=False):
    if K.image_dim_ordering() == "th":
        channel_axis = 1
    else:
        channel_axis = -1

    x = Convolution2D(nb_filter, nb_row, nb_col, subsample=subsample, border_mode=border_mode, bias=bias)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    return x


def inception_stem(input):
    channel_axis = -1
    # Input Shape is 299 x 299 x 3 (th) or 3 x 299 x 299 (th)
    x = conv_block(input, 32, 3, 3, subsample=(2, 2), border_mode='valid')
    x = conv_block(x, 32, 3, 3, border_mode='valid')
    x = conv_block(x, 64, 3, 3)

    x1 = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid')(x)
    x2 = conv_block(x, 96, 3, 3, subsample=(2, 2), border_mode='valid')
    x = concatenate([x1, x2], axis=channel_axis)
    #x = merge([x1, x2], mode='concat', concat_axis=channel_axis)

    x1 = conv_block(x, 64, 1, 1)
    x1 = conv_block(x1, 96, 3, 3, border_mode='valid')

    x2 = conv_block(x, 64, 1, 1)
    x2 = conv_block(x2, 64, 1, 7)
    x2 = conv_block(x2, 64, 7, 1)
    x2 = conv_block(x2, 96, 3, 3, border_mode='valid')

    #x = merge([x1, x2], mode='concat', concat_axis=channel_axis)
    x = concatenate([x1, x2], axis=channel_axis)

    x1 = conv_block(x, 192, 3, 3, subsample=(2, 2), border_mode='valid')
    x2 = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid')(x)

    #x = merge([x1, x2], mode='concat', concat_axis=channel_axis)
    x = concatenate([x1, x2], axis=channel_axis)

    return x


def inception_A(input):
    channel_axis = -1

    a1 = conv_block(input, 96, 1, 1)

    a2 = conv_block(input, 64, 1, 1)
    a2 = conv_block(a2, 96, 3, 3)

    a3 = conv_block(input, 64, 1, 1)
    a3 = conv_block(a3, 96, 3, 3)
    a3 = conv_block(a3, 96, 3, 3)

    a4 = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(input)
    a4 = conv_block(a4, 96, 1, 1)

   # m = merge([a1, a2, a3, a4], mode='concat', concat_axis=channel_axis)
    m = concatenate([a1, a2, a3, a4], axis=channel_axis)

    return m


def inception_B(input):
    channel_axis = -1

    b1 = conv_block(input, 384, 1, 1)

    b2 = conv_block(input, 192, 1, 1)
    b2 = conv_block(b2, 224, 1, 7)
    b2 = conv_block(b2, 256, 7, 1)

    b3 = conv_block(input, 192, 1, 1)
    b3 = conv_block(b3, 192, 7, 1)
    b3 = conv_block(b3, 224, 1, 7)
    b3 = conv_block(b3, 224, 7, 1)
    b3 = conv_block(b3, 256, 1, 7)

    b4 = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(input)
    b4 = conv_block(b4, 128, 1, 1)

    #m = merge([b1, b2, b3, b4], mode='concat', concat_axis=channel_axis)
    m = concatenate([b1, b2, b3, b4], axis=channel_axis)

    return m


def inception_C(input):
    channel_axis = -1

    c1 = conv_block(input, 256, 1, 1)

    c2 = conv_block(input, 384, 1, 1)
    c2_1 = conv_block(c2, 256, 1, 3)
    c2_2 = conv_block(c2, 256, 3, 1)
   # c2 = merge([c2_1, c2_2], mode='concat', concat_axis=channel_axis)
    c2 = concatenate([c2_1, c2_2], axis=channel_axis)


    c3 = conv_block(input, 384, 1, 1)
    c3 = conv_block(c3, 448, 3, 1)
    c3 = conv_block(c3, 512, 1, 3)
    c3_1 = conv_block(c3, 256, 1, 3)
    c3_2 = conv_block(c3, 256, 3, 1)
    #c3 = merge([c3_1, c3_2], mode='concat', concat_axis=channel_axis)
    c3 = concatenate([c3_1, c3_2], axis=channel_axis)

    c4 = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(input)
    c4 = conv_block(c4, 256, 1, 1)

   # m = merge([c1, c2, c3, c4], mode='concat', concat_axis=channel_axis)
    m = concatenate([c1, c2, c3, c4], axis=channel_axis)

    return m


def reduction_A(input):
    channel_axis = -1

    r1 = conv_block(input, 384, 3, 3, subsample=(2, 2), border_mode='valid')

    r2 = conv_block(input, 192, 1, 1)
    r2 = conv_block(r2, 224, 3, 3)
    r2 = conv_block(r2, 256, 3, 3, subsample=(2, 2), border_mode='valid')

    r3 = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid')(input)

    #m = merge([r1, r2, r3], mode='concat', concat_axis=channel_axis)
    m = concatenate([r1, r2, r3], axis=channel_axis)

    return m


def reduction_B(input):
    channel_axis = -1

    r1 = conv_block(input, 192, 1, 1)
    r1 = conv_block(r1, 192, 3, 3, subsample=(2, 2), border_mode='valid')

    r2 = conv_block(input, 256, 1, 1)
    r2 = conv_block(r2, 256, 1, 7)
    r2 = conv_block(r2, 320, 7, 1)
    r2 = conv_block(r2, 320, 3, 3, subsample=(2, 2), border_mode='valid')

    r3 = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid')(input)

    #m = merge([r1, r2, r3], mode='concat', concat_axis=channel_axis)
    m = concatenate([r1, r2, r3], axis=channel_axis)

    return m

# nb_classes is number of classes, you should change it according to your dataset.

def create_inception_v4(nb_classes=26, load_weights=False):
    '''
    Creates a inception v4 network
    :param nb_classes: number of classes.txt
    :return: Keras Model with 1 input and 1 output
    '''

    init = Input((299,299, 3))

    x = inception_stem(init)

    # 4 x Inception A
    for i in range(4):
        x = inception_A(x)

    # Reduction A
    x = reduction_A(x)

    # 7 x Inception B
    for i in range(7):
        x = inception_B(x)

    # Reduction B
    x = reduction_B(x)

    # 3 x Inception C
    for i in range(3):
        x = inception_C(x)

    # Average Pooling
    x = AveragePooling2D((8, 8))(x)

    # Dropout
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    # Output
    out = Dense(output_dim=nb_classes, activation='softmax')(x)

    model = Model(init, out, name='Inception-v4')

    if load_weights:
        weights = checkpoint_path
        model.load_weights(weights)
        print("Model weights loaded.")

    return model

model = create_inception_v4(load_weights=False)
model = multi_gpu_model(model, gpus=4)
model.summary()


train_dir = 'train/'
val_dir = 'validation/'


sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

datagen=ImageDataGenerator(rescale=1/255,
            rotation_range=40,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest',
            samplewise_std_normalization=True)

val_datagen = ImageDataGenerator(rescale=1/255)

train_generator = datagen.flow_from_directory(train_dir,target_size=(299,299),class_mode="categorical")
val_gen = datagen.flow_from_directory(val_dir,target_size=(299,299),class_mode="categorical")

mc = keras.callbacks.ModelCheckpoint("checkpoints/inceptionv4.h5",save_best_only=True, save_weights_only=True)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True), metrics=["accuracy"])
hist = model.fit_generator(train_generator,steps_per_epoch=500,epochs=100,verbose=True,validation_data=val_gen,validation_steps=10,callbacks=[mc])

### After model finishes the training, test it with custom image

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

model=load_model('checkpoints/inceptionv4.h5')
test_accuracy=model.evaluate_generator(val_gen,steps=25)
print(test_accuracy)

In [0]:
classes = ['1', '2', '3']
img_width, img_height = 299, 299

img = image.load_img('test.png', target_size=(img_width, img_height))
test_image = image.img_to_array(img)
test_image.shape
img = np.expand_dims(test_image, axis=0)
classes = model.predict(images, batch_size=10)
result = model.predict_classes
print(result)